In [1]:
from google.cloud import speech

audiofile = "gs://coreymaher-medical-transcription/original_audio/car0001-short.mp3"

/Users/coreymaher/.virtualenvs/ambient-listening/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [10]:
def transcribe(audiofile):
    # Instantiates a client
    client = speech.SpeechClient()

    # The name of the audio file to transcribe
    gcs_uri = audiofile

    audio = speech.RecognitionAudio(uri=gcs_uri)

    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.MP3,
        sample_rate_hertz=16000,
        language_code="en-US",
        model="medical_conversation"
    )

    # Detects speech in the audio file
    response = client.recognize(config=config, audio=audio)
    return response

In [7]:
def transcribe_gcs(gcs_uri: str) -> str:
    """Asynchronously transcribes the audio file specified by the gcs_uri.

    Args:
        gcs_uri: The Google Cloud Storage path to an audio file.

    Returns:
        The generated transcript from the audio file provided.
    """
    from google.cloud import speech

    client = speech.SpeechClient()

    audio = speech.RecognitionAudio(uri=gcs_uri)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.MP3,
        sample_rate_hertz=16000,
        language_code="en-US",
        model="medical_conversation"
    )

    operation = client.long_running_recognize(config=config, audio=audio)

    print("Waiting for operation to complete...")
    response = operation.result(timeout=180)

    transcript_builder = []
    # Each result is for a consecutive portion of the audio. Iterate through
    # them to get the transcripts for the entire audio file.
    for result in response.results:
        # The first alternative is the most likely one for this portion.
        transcript_builder.append(f"\nTranscript: {result.alternatives[0].transcript}")
        transcript_builder.append(f"\nConfidence: {result.alternatives[0].confidence}")

    transcript = "".join(transcript_builder)
    # print(transcript)

    return transcript

In [9]:
from google.cloud import storage




        
    # ... rest of your transcription code using the 'content' variable


In [11]:
from google.cloud import speech
client = speech.SpeechClient()
config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=48000,
        language_code="en-US",
        model="medical_conversation",
    )

In [31]:
def audio_stream_generator(uri, chunk_size=8000):
    if uri[0:4] == "gs://" :
        storage_client = storage.Client()
        blob = storage.Blob.from_string(uri, storage_client)
        with blob.open("rb") as audio_file:
            data = audio_file.read(chunk_size)  # Read the first chunk
            while len(data) > 0:                       # Check if all data was read
                yield data                    # Yield the chunk
                data = audio_file.read(chunk_size) # Read the next chunk
    else: 
        with open(uri, "rb") as audio_file:
            data = audio_file.read(chunk_size)  # Read the first chunk
            while len(data) > 0:                       # Check if all data was read
                yield data                    # Yield the chunk
                data = audio_file.read(chunk_size) # Read the next chunk

In [32]:
streaming_config = speech.StreamingRecognitionConfig(config=config)

requests = (
    speech.StreamingRecognizeRequest(audio_content=content) for content in audio_stream_generator(
        'sample_wavs/GAS0007.wav', 24000)
        )

responses = client.streaming_recognize(streaming_config, requests)

for response in responses:
        # Once the transcription has settled, the first result will contain the
        # is_final result. The other results will be for subsequent portions of
        # the audio.
    for result in response.results:
        # print(f"Finished: {result.is_final}")
        # print(f"Stability: {result.stability}")
        alternatives = result.alternatives
        # The alternatives are ordered from most likely to least.
        for alternative in alternatives:
            # print(f"Confidence: {alternative.confidence}")
            print(f"Transcript: {alternative.transcript}")

Transcript: So , I understand you've been experiencing some abdominal pain ?  A stomach hurt . It started hurting more the last couple of days , maybe three days ago
Transcript:  So , for the last three days , you said it's been hurting more .  Um-hum .  Did you have pain before that ?  Um-hum . It felt weird , like , crampy , but I thought I was constipated , because I haven't been able to go to the bathroom as well .  Hum .  I just , I was kind of bloated a bit , but , yeah , about , about that time , it started to feel a little bit more painful .  Okay . Um , can you tell me where you're feeling the pain the most , ?  Yeah , kind of near my right hip , lower , where my stomach is , right and below my bellybutton . Sometimes , a little bit on the top , just below my belly button , but mainly , just not right lower side , yeah .  Okay . Have you ever had pain like this in the past
Transcript:  ?  No . Never
Transcript:  And for the last three days , has it been constant ? Has it been 

In [19]:
def streamingtranscribe(content):
    client = speech.SpeechClient()
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.MP3,
        sample_rate_hertz=16000,
        language_code="en-US",
        model="medical_conversation",
    )
    streaming_config = speech.StreamingRecognitionConfig(config=config)

    requests = (speech.StreamingRecognizeRequest(audio_content=content) for content in generate_chunks(content))

    responses = client.streaming_recognize(streaming_config, requests)

    for response in responses:
        for result in response.results:
            # First alternative is the most probable result
            alternatives = result.alternatives
            for alternative in alternatives: 
                yield alternative.transcript

In [20]:
for t in streamingtranscribe(content):
    print(t)

Uh , brought you in today .  Sure . Um , I'm just having a lot of chest pain , um , and , so , I thought I should get it checked out .  Uh , okay . Uh , and before we , uh , start , could you , uh , remind me of your , uh , gender and age ?  Sure . Um , thirty nine , uh , I'm a male .  Okay . Um , , and so , when did this chest pain start ?  Um , it started last night , , uh , but it's becoming sharper .  Okay . And , uh , where


In [18]:
next(streamingtranscribe(content))

"Uh , brought you in today .  Sure . Um , I'm just having a lot of chest pain , um , and , so , I thought I should get it checked out .  Uh , okay . Uh , and before we , uh , start , could you , uh , remind me of your , uh , gender and age ?  Sure . Um , thirty nine , uh , I'm a male .  Okay . Um , , and so , when did this chest pain start ?  Um , it started last night , , uh , but it's becoming sharper .  Okay . And , uh , where"

In [23]:
len(content)

820254